In [1]:
import numpy as np
import scipy.stats
import torch
from botorch import acquisition, sampling
from botorch.acquisition.objective import ScalarizedPosteriorTransform, ConstrainedMCObjective
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize
from botorch.optim import optimize_acqf
from botorch.utils.multi_objective.box_decompositions import FastNondominatedPartitioning
from gpytorch.mlls import ExactMarginalLogLikelihood
from tqdm.notebook import trange

device = "cuda:0"
torch.set_default_device(device)

def array(x: torch.Tensor) -> np.ndarray:
    return x.detach().cpu().numpy()

C:\Users\alexa\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
default_a = 15.0 
default_b = 2.0 
default_noise = 0.2

def example_1d_function(
    x: torch.Tensor, a: float = default_a, b: float = default_b
) -> torch.Tensor:
    """Ground truth function.""" 
    return torch.sin(a * x) + b * x**2

def example_noisy_1d_function(
    x: torch.Tensor, a: float = default_a, b: float = default_b, noise: float = default_noise
) -> torch.Tensor:
    """Noisy ground truth observations."""
    return example_1d_function(x, a, b) + noise * torch.randn_like(x)